In this notebook, we explore the baseline measures for navground simulations to see how well these differenciate between behaviours. 
These measures are the safety margin violations, collisions, agent movement efficacy and deadlocks.

We focus on the cross scenario.

In [ ]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
import scipy.spatial.distance as dist
from navground import sim, core
from tslearn.generators import random_walks
from tslearn import metrics
import seaborn as sns
import gudhi
from gudhi.wasserstein import wasserstein_distance
from navground.sim.ui.video import display_video_from_run, record_video_from_run
from functools import partial
from IPython.display import Image

# Create folder to save bottleneck distances between matching diagrams
import os
os.makedirs("Baseline_cross_matrices", exist_ok=True)
os.makedirs("plots", exist_ok=True)

In [ ]:
parser = argparse.ArgumentParser(description='Simulation Parameters')
parser.add_argument('--side', type=float, default=10.0, help='Side of the environment square')
parser.add_argument('--num_runs', type=int, default=1, help='Number of simulation runs')
parser.add_argument('--num_steps', type=int, default=100, help='Number of steps in the simulation')
parser.add_argument('--time_step', type=float, default=0.1, help='Time step for the simulation')
parser.add_argument('--num_agents', type=int, default=10, help='Number of agents in the simulation')
parser.add_argument('--max_speed', type=float, default=1.0, help='Maximum speed of agents')
parser.add_argument('--optimal_speed', type=float, default=1.0, help='Maximum speed of agents')
parser.add_argument('--radius', type=float, default=0.25, help='Radius of agents')
parser.add_argument('--safety_margin', type=float, default=0.1, help='Safety margin for agents')
parser.add_argument('--behavior', type=str, default='HL', help='Behavior type')
parser.add_argument('--max_edge_length', type=float, default=100.0, help='Maximum edge length in the simplicial complex')
parser.add_argument('--time_delay', type=int, default=1, help='Time delay to analise simulation intervals')
parser.add_argument('--embedding_length', type=int, default=10, help='Length of the simulation intervals')
parser.add_argument('--epsilon', type=int, default=50, help='time differences for matching and bottleneck distance computation')

In [ ]:
args = parser.parse_args([
        '--side', '10.0',
        '--num_runs', '12',
        '--num_steps', '600',
        '--time_step', '0.1',
        '--num_agents', '10',
        '--max_speed', '1.66',
        '--optimal_speed', '1.2',
        '--radius', '0.4',
        '--safety_margin', '0.1',
        '--behavior', 'HL',
        '--max_edge_length', '33.0',
        '--time_delay', '5',
        '--embedding_length', '11',
        '--epsilon', '50',
    ])

## 1. HL behavior

We run the experiment with HL behaviour.

In [ ]:
yaml = f"""
runs: {args.num_runs}
steps: {args.num_steps}
time_step: {args.time_step}
save_directory: ''
record_pose: true
record_twist: true
record_collisions: true
record_deadlocks: true
record_safety_violation: true
record_efficacy: true
scenario:
  type: Cross
  side: {args.side}
  tolerance: 1
  groups:
    -
      type: thymio
      number: {args.num_agents}
      radius: {args.radius}
      control_period: 0.1
      speed_tolerance: 0.02
      kinematics:
        type: 2WDiff
        wheel_axis: 0.094
        max_speed: {args.max_speed}
      behavior:
        type: {args.behavior}
        optimal_speed:
            sampler: uniform
            from: {args.optimal_speed*0.8}
            to: {args.optimal_speed*1.2}
        horizon: 5.0
        safety_margin: {args.safety_margin}
      state_estimation:
        type: Bounded
        range: 5.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()
runs = experiment.runs

In [ ]:
display_video_from_run(run=runs[1], factor=3.0, fps=30)

In [ ]:
experiment.runs[0].collisions

In [ ]:
experiment.runs[0].safety_violations[145]

In [ ]:
experiment.runs[0].safety_violations.shape

In [ ]:
safety_violations_HL = []
for run_key in experiment.runs.keys():
    run = experiment.runs[run_key]
    safety_violations_HL.append(np.sum(run.safety_violations, axis=1))
# end for
safety_violations_HL = np.array(safety_violations_HL)

In [ ]:
fig, ax = plt.subplots(figsize=(16,5))
signal = safety_violations_HL[1]
time = range(len(signal))
ax.plot(time, signal)
ax.set_ylim([0,1])

## 2. ORCA behavior

We run the experiment with ORCA behaviour.

In [ ]:
args.behavior = "ORCA"

yaml = f"""
runs: {args.num_runs}
steps: {args.num_steps}
time_step: {args.time_step}
save_directory: ''
record_pose: true
record_twist: true
record_collisions: true
record_deadlocks: true
record_safety_violation: true
record_efficacy: true
scenario:
  type: Cross
  side: {args.side}
  tolerance: 1
  groups:
    -
      type: thymio
      number: {args.num_agents}
      radius: {args.radius}
      control_period: 0.1
      speed_tolerance: 0.02
      kinematics:
        type: 2WDiff
        wheel_axis: 0.094
        max_speed: {args.max_speed}
      behavior:
        type: {args.behavior}
        optimal_speed:
            sampler: uniform
            from: {args.optimal_speed*0.8}
            to: {args.optimal_speed*1.2}
        horizon: 5.0
        safety_margin: {args.safety_margin}
      state_estimation:
        type: Bounded
        range: 5.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()
runs = experiment.runs

In [ ]:
safety_violations_ORCA = []
for run_key in experiment.runs.keys():
    run = experiment.runs[run_key]
    safety_violations_ORCA.append(np.sum(run.safety_violations, axis=1))
# end for
safety_violations_ORCA = np.array(safety_violations_ORCA)

In [ ]:
fig, ax = plt.subplots(figsize=(16,5))
signal = safety_violations_ORCA[6]
time = range(len(signal))
ax.plot(time, signal)
ax.set_ylim([0,1])

## 3. SocialForce behavior

We run the experiment with SF behaviour.

In [ ]:
args.behavior = "SocialForce"

yaml = f"""
runs: {args.num_runs}
steps: {args.num_steps}
time_step: {args.time_step}
save_directory: ''
record_pose: true
record_twist: true
record_collisions: true
record_deadlocks: true
record_safety_violation: true
record_efficacy: true
scenario:
  type: Cross
  side: {args.side}
  tolerance: 1
  groups:
    -
      type: thymio
      number: {args.num_agents}
      radius: {args.radius}
      control_period: 0.1
      speed_tolerance: 0.02
      kinematics:
        type: 2WDiff
        wheel_axis: 0.094
        max_speed: {args.max_speed}
      behavior:
        type: {args.behavior}
        optimal_speed:
            sampler: uniform
            from: {args.optimal_speed*0.8}
            to: {args.optimal_speed*1.2}
        horizon: 5.0
        safety_margin: {args.safety_margin}
      state_estimation:
        type: Bounded
        range: 5.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()
runs = experiment.runs

In [ ]:
safety_violations_SF = []
for run_key in experiment.runs.keys():
    run = experiment.runs[run_key]
    safety_violations_SF.append(np.sum(run.safety_violations, axis=1))
# end for
safety_violations_SF = np.array(safety_violations_SF)

In [ ]:
fig, ax = plt.subplots(figsize=(16,5))
signal = safety_violations_SF[1]
time = range(len(signal))
ax.plot(time, signal)
ax.set_ylim([0,1])

In [ ]:
all_safety_violations = np.vstack([
    safety_violations_HL, safety_violations_ORCA, safety_violations_SF
])
all_safety_violations.shape

In [ ]:
import umap

reducer = umap.UMAP()

embedding = reducer.fit_transform(all_safety_violations)
fig, ax = plt.subplots(figsize=(5,5))
embedding_dict = {}
behaviour_list = ["HL", "ORCA", "SF"]
for i, behaviour in enumerate(behaviour_list):
    embedding_dict[behaviour] = embedding[args.num_runs*i:args.num_runs*(i+1)]
# end for
color_behaviour = { "HL": "blue", "ORCA": "red", "SF": "green"}
for behaviour in behaviour_list:
    ax.scatter(embedding_dict[behaviour][:,0], embedding_dict[behaviour][:,1], color=color_behaviour[behaviour], label=behaviour)

plt.legend()
plt.savefig(os.path.join("plots", "UMAP-projection.png"))